<a href="https://colab.research.google.com/github/Georgeman31/std_projects/blob/main/NLP/Transfer_learning_ProtasovEA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка данных

In [ ]:
# upload kaggle API to collab correctly
from google.colab import files, drive
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [ ]:
import os
import shutil

path = "/content/.kaggle"
os.mkdir(path)
shutil.move("/content/kaggle.json", "/content/.kaggle/kaggle.json")

'/content/.kaggle/kaggle.json'

In [ ]:
!mv .kaggle /root/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# download data
!kaggle competitions download -c nlp-txt-classification

 73% 3.00M/4.08M [00:00<00:00, 5.61MB/s]
100% 4.08M/4.08M [00:00<00:00, 6.11MB/s]


# Предобработка данных

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
# импорт используемых библиотек
import pandas as pd
import numpy as np
import re

import datasets # обработка датасетов
from datasets import Dataset, load_dataset

from sklearn.model_selection import train_test_split # предобработка
from sklearn import preprocessing

import evaluate # метрики

import transformers # построение модели трансформера
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer
from transformers import TrainingArguments
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    default_data_collator   
)

Получение данных

In [ ]:
!unzip nlp-txt-classification.zip

Archive:  nlp-txt-classification.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# чтение файлов
test_data = pd.read_csv("test.csv")
train_data = pd.read_csv("train.csv")
submission_data = pd.read_csv("sample_submission.csv")

In [ ]:
train_data

,Unnamed: 0,Text,Sentiment
0,0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,1,advice Talk to your neighbours family to excha...,Positive
2,2,Coronavirus Australia: Woolworths to give elde...,Positive
3,3,My food stock is not the only one which is emp...,Positive
4,4,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...
41154,41152,Airline pilots offering to stock supermarket s...,Neutral
41155,41153,Response to complaint not provided citing COVI...,Extremely Negative
41156,41154,You know itÂs getting tough when @KameronWild...,Positive
41157,41155,Is it wrong that the smell of hand sanitizer i...,Neutral


In [ ]:
test_data

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,TRENDING: New Yorkers encounter empty supermar...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,When I couldn't find hand sanitizer at Fred Me...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,Find out how you can protect yourself and love...
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,#Panic buying hits #NewYork City as anxious sh...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,#toiletpaper #dunnypaper #coronavirus #coronav...
...,...,...
3793,65712d27-5c41-4863-b74f-0bd66199b7df,Meanwhile In A Supermarket in Israel -- People...
3794,9fd189c5-e79c-49d7-8985-576450a4e6e3,Did you panic buy a lot of non-perishable item...
3795,3a06785f-6f9b-4f4d-9880-22562ad3e296,Asst Prof of Economics @cconces was on @NBCPhi...
3796,dd29ff09-9bc2-40f4-8201-4b6361aca760,Gov need to do somethings instead of biar je r...


In [ ]:
submission_data

,id,Sentiment
0,787bc85b-20d4-46d8-84a0-562a2527f684,Neutral
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,Neutral
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,Neutral
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,Neutral
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,Neutral
...,...,...
3793,65712d27-5c41-4863-b74f-0bd66199b7df,Neutral
3794,9fd189c5-e79c-49d7-8985-576450a4e6e3,Neutral
3795,3a06785f-6f9b-4f4d-9880-22562ad3e296,Neutral
3796,dd29ff09-9bc2-40f4-8201-4b6361aca760,Neutral


In [ ]:
def text_cleaner(text, remove_digits=True):
    '''Очистка текста от лишних символов'''
    
    text = re.sub('http://\S+|https://\S+|www.\S+', '', text)
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text.strip()

In [ ]:
train_data = train_data.drop(train_data.columns[0],axis=1)
train_data = train_data.dropna()

In [ ]:
train_data['Text'] = train_data['Text'].apply(text_cleaner)

In [ ]:
# препроцессинг лейблов
label_encoder = preprocessing.LabelEncoder()

train_data['Sentiment'] = label_encoder.fit_transform(train_data['Sentiment'])

train_data['Sentiment'].unique()
print(label_encoder.classes_)

['Extremely Negative' 'Extremely Positive' 'Negative' 'Neutral' 'Positive']


In [ ]:
# проставление классов лейблам
lst1 = list(label_encoder.classes_)
lst2 = [3, 4, 0, 2, 1]
label_to_id= dict(zip(lst1, lst2))

In [ ]:
label_to_id

{'Extremely Negative': 3,
 'Extremely Positive': 4,
 'Negative': 0,
 'Neutral': 2,
 'Positive': 1}

In [ ]:
test_data = test_data.dropna()

In [ ]:
train_data = train_data.rename(columns={'Text':'text', 'Sentiment':'label'})

In [ ]:
test_data['id1'] = test_data['id']
test_data = test_data.drop('id', axis=1)
test_data = test_data.rename(columns={'Text':'text', 'id1':'id'})

In [ ]:
# преобразование типов данных датасетов в dataset
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 41155
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'id', '__index_level_0__'],
    num_rows: 3798
})

# Подготовка модели

Токенизация

In [ ]:
checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [ ]:
tokenized_datasets = train_dataset.map(tokenize_function)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets

  0%|          | 0/41155 [00:00<?, ?ex/s]

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41155
})

Загрузка предобученной модели BERT

In [ ]:
config = AutoConfig.from_pretrained("bert-base-uncased", num_labels=5)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",config=config,ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Функция для расчета метрик

In [ ]:
def metrics(eval_preds):
    '''Расчет метрик качества для модели'''
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Настройка модели

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
model.config.label2id = label_to_id
model.config.id2label = {id: label for label, id in config.label2id.items()}

In [ ]:
tokenized_datasets

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 41155
})

In [ ]:
split_data= train_dataset.train_test_split(test_size=0.2)
split_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 32924
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 8231
    })
})

In [ ]:
tokenized_train = split_data['train'].map(tokenize_function, batched=True)
tokenized_val = split_data['test'].map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/33 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
# проставление параметров модели
training_args = TrainingArguments(
    "test-trainer",     
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=1,
    optim='adamw_torch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=metrics
)

Обучение

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 41155
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3861
  Number of trainable parameters = 109486085
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.401700,0.409593,0.857004
2,0.156400,0.226445,0.928198
3,0.010100,0.130818,0.957478


Saving model checkpoint to test-trainer/checkpoint-500
Configuration saved in test-trainer/checkpoint-500/config.json
Model weights saved in test-trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-1000
Configuration saved in test-trainer/checkpoint-1000/config.json
Model weights saved in test-trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1000/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
*

Saving model checkpoint to test-trainer/checkpoint-1500
Configuration saved in test-trainer/checkpoint-1500/config.json
Model weights saved in test-trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2000
Configuration saved in test-trainer/checkpoint-2000/config.json
Model weights saved in test-trainer/checkpoint-2000/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2000/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-2000/special_tokens_map.json
Saving model checkpoint to test-trainer/checkpoint-2500
Configuration saved in test-trainer/checkpoint-2500/config.json
Model weights saved in test-trainer/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-2500/tokenizer_config.json
Special tokens file 

TrainOutput(global_step=3861, training_loss=0.5070315444435779, metrics={'train_runtime': 1507.0735, 'train_samples_per_second': 81.924, 'train_steps_per_second': 2.562, 'total_flos': 3920449530758538.0, 'train_loss': 0.5070315444435779, 'epoch': 3.0})

Предикты

In [ ]:
def get_prediction(text):
    '''Получение предсказаний'''
    inputs = tokenizer(text, truncation=True,padding=True, return_tensors="pt").to("cuda")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return model.config.id2label[probs.argmax().item()]

In [ ]:
tokenized_test = test_dataset.map(tokenize_function, batched=True)
trainer.evaluate(tokenized_test)

  0%|          | 0/4 [00:00<?, ?ba/s]

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, __index_level_0__, text. If id, __index_level_0__, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3798
  Batch size = 16


{'eval_runtime': 20.0659,
 'eval_samples_per_second': 189.276,
 'eval_steps_per_second': 11.861,
 'epoch': 3.0}

In [ ]:
test_data['Sentiment'] = test_data['text'].apply(lambda text: get_prediction(text))
test_data.head()

,text,id,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,787bc85b-20d4-46d8-84a0-562a2527f684,Neutral
1,When I couldn't find hand sanitizer at Fred Me...,17e934cd-ba94-4d4f-9ac0-ead202abe241,Extremely Positive
2,Find out how you can protect yourself and love...,5914534b-2b0f-4de8-bb8a-e25587697e0d,Positive
3,#Panic buying hits #NewYork City as anxious sh...,cdf06cfe-29ae-48ee-ac6d-be448103ba45,Neutral
4,#toiletpaper #dunnypaper #coronavirus #coronav...,aff63979-0256-4fb9-a2d9-86a3d3ca5470,Extremely Negative


Kaggle submit

In [ ]:
submission = test_data[['id', 'Sentiment']]
submission.to_csv(('sub_to_kaggle1.csv'), index=False)